# Compute i coefficients

# Define symbolic

In [1]:
import brightway2 as bw
import pandas as pd
import os

# Set working directry
path = "."
os.chdir(path)

# Local functions
from utils.lookup_func import lookup_geothermal

# Set project
bw.projects.set_current("Geothermal")

# Retrieve activities
wellhead, diesel, steel, cement, water, \
drilling_mud, drill_wst, wells_closr, coll_pipe, \
plant, ORC_fluid, ORC_fluid_wst, diesel_stim, co2,_, _ = lookup_geothermal()
cooling_tower=bw.Database("geothermal energy").search("cooling tower")[0].key

list_act = [wellhead, diesel, steel, cement, water, 
         drilling_mud, drill_wst, wells_closr, coll_pipe,
         plant, cooling_tower, ORC_fluid, ORC_fluid_wst, diesel_stim]

# Retrieve methods 
ILCD_CC = [method for method in bw.methods if "ILCD 2.0 2018 midpoint no LT" in str(method) and "climate change total" in str(method)]
ILCD_HH = [method for method in bw.methods if "ILCD 2.0 2018 midpoint no LT" in str(method) and "human health" in str(method)]
ILCD_EQ = [method for method in bw.methods if "ILCD 2.0 2018 midpoint no LT" in str(method) and "ecosystem quality" in str(method)]
ILCD_RE = [method for method in bw.methods if "ILCD 2.0 2018 midpoint no LT" in str(method) and "resources" in str(method)]
ILCD = ILCD_CC + ILCD_HH + ILCD_EQ + ILCD_RE

# Calculate impact of activities
lca = bw.LCA({list_act[0]: 1}, ILCD[0])
lca.lci()
lca.lcia()
coeff = {}
for method in ILCD:
    s=[]
    lca.switch_method(method)
    for act in list_act:
        lca.redo_lcia({act: 1})
        s.append(lca.score)
    coeff[method] = s

# Retrieve CF for co2 emissions    
for method in ILCD:
    CFs = bw.Method(method).load()
    coeff[method].append(next((cf[1] for cf in CFs if cf[0]==co2),0))

# Build matrix
col_names = ["wellhead", "diesel", "steel", "cement", "water", \
         "drilling_mud", "drill_wst", "wells_closr", "coll_pipe", \
         "plant", "cooling tower", "ORC_fluid", "ORC_fluid_wst", "diesel_stim", "co2"]
coeff_matrix = pd.DataFrame.from_dict(coeff, orient="index", columns=col_names)

# Re-arrange matrix
coeff_matrix["concrete"] = coeff_matrix["cement"] + coeff_matrix["water"] * 1/0.65
coeff_matrix["ORC_fluid_tot"] = coeff_matrix["ORC_fluid"] - coeff_matrix["ORC_fluid_wst"]
coeff_matrix["electricity_stim"] = coeff_matrix["diesel_stim"] * 3.6
coeff_matrix["drill_wst"] = coeff_matrix["drill_wst"] * -1
coeff_matrix=coeff_matrix.drop(columns=["cement", "ORC_fluid", "ORC_fluid_wst", "diesel_stim"])

col_ord = ["wellhead", "diesel", "steel", "concrete", \
         "drilling_mud", "drill_wst", "wells_closr", "coll_pipe", \
         "plant", "cooling tower", "ORC_fluid_tot", "water", "electricity_stim", "co2"]
coeff_matrix=coeff_matrix[col_ord]

is_= ["i1", "i2.1", "i2.2", "i2.3", "i2.4", "i2.5", \
      "i2.6","i3", "i4.1", "i4.2", "i4.3", "i5.1", "i5.2", "i6"]

if len(coeff_matrix.columns) == len(is_):
    coeff_matrix.columns=is_

In [2]:
from sympy import symbols, collect, ratsimp, fraction

In [3]:
# Coefficients
i1, i2_1, i2_2, i2_3, i2_4, i2_5, i2_6, i3, i4, i4_1, i4_2, i4_3, i5_1, i5_2, i6 = \
symbols('i1, i2_1, i2_2, i2_3, i2_4, i2_5, i2_6, i3, i4, i4_1, i4_2, i4_3, i5_1, i5_2, i6')

# Other variables that are actually irrelevant, because they will be multiplied by 0 in the enhanced model
# I wanna keep them though for the sake of completeness
SW_n, S_w, S_el = symbols('SW_n, SW, S_el')

# Main parameters
P_ne, CW_ne, PIratio, W_d, E_co2, SR_p, D_i, LT, SR_m, SR_e, D, C_S, C_C, DM, CP, CF, AP = \
symbols('P_ne, CW_ne, PIratio, W_d, E_co2, SR_p, D_i, LT, SR_m, SR_e, D, C_S, C_C, DM, CP, CF, AP')

# Constants
CT_el, CT_n, W_en, OF,DW = symbols ('CT_el, CT_n, W_E_en, OF, DW')


# Main equation

In [4]:
# Supporting equations for CONVENTIONAL geothermal
W_Pn = P_ne/CW_ne             # production wells
W_In = P_ne/CW_ne / PIratio  # injection wells
W_Mn = W_Pn*D_i*LT            # makeup wells

# Number of wells with success rate
W_n_sr = W_Pn/(SR_p/100) + W_In/(SR_p/100) + W_Mn/(SR_m/100)
W_n = W_Pn + W_In + W_Mn
W_E_en = W_en
W_E_en_sr = W_en/(SR_e/100)

In [5]:
# Equation
nominator = (W_n + W_E_en) *i1 \
          + W_d* (W_n_sr + W_E_en_sr) * (D*i2_1 + C_S*i2_2 + C_C*i2_3  +  DM*i2_4 + DW*i2_5 + i2_6) + \
          + W_n*CP*i3 \
          + P_ne*i4 \
          + SW_n*S_w * (i5_1 + S_el*i5_2)

c1 = P_ne*CF*(1-AP)*LT*8760000
#d1 = CT_el*CT_n*1000*LT
denominator = c1 #  -  d1

summand = E_co2*i6

eq = nominator/denominator + summand
eq = eq.subs(i4, i4_1 + i4_2*CT_n + i4_3*OF)

In [6]:
# Show equation
eq

E_co2*i6 + (CP*i3*(D_i*LT*P_ne/CW_ne + P_ne/CW_ne + P_ne/(CW_ne*PIratio)) + P_ne*(CT_n*i4_2 + OF*i4_3 + i4_1) + SW*SW_n*(S_el*i5_2 + i5_1) + W_d*(100*W_E_en/SR_e + 100*D_i*LT*P_ne/(CW_ne*SR_m) + 100*P_ne/(CW_ne*SR_p) + 100*P_ne/(CW_ne*PIratio*SR_p))*(C_C*i2_3 + C_S*i2_2 + D*i2_1 + DM*i2_4 + DW*i2_5 + i2_6) + i1*(W_E_en + D_i*LT*P_ne/CW_ne + P_ne/CW_ne + P_ne/(CW_ne*PIratio)))/(8760000*CF*LT*P_ne*(1 - AP))

# Get parameters values

In [58]:
success_rate = False # For TWI software

In [59]:
from cge_klausen import get_parameters
parameters=get_parameters()
parameters.static()    

# Rename parameters (in accordance with equation)

# Parameters of simplified models
CW_ne_val    = parameters["gross_power_per_well"]
E_co2_val    = parameters["co2_emissions"]
D_i_val      = parameters["initial_harmonic_decline_rate"]
if success_rate == True:
    SR_p_val     = parameters["success_rate_primary_wells"]
elif success_rate == False:
    SR_p_val     = 100
W_d_val      = parameters["average_depth_of_wells"]

# Parameters to be fixed
P_ne_val     = parameters["installed_capacity"]
PIratio_val  = parameters["production_to_injection_ratio"]
LT_val       = parameters["lifetime"]
if success_rate == True:
    SR_m_val     = parameters["success_rate_makeup_wells"]   
    SR_e_val     = parameters["success_rate_exploration_wells"]
elif success_rate == False:
    SR_m_val     = 100
    SR_e_val     = 100
D_val        = parameters["specific_diesel_consumption"]
C_S_val      = parameters["specific_steel_consumption"]
C_C_val      = parameters["specific_cement_consumption"]
DM_val       = parameters["specific_drilling_mud_consumption"]
CP_val       = parameters["collection_pipelines"]
CF_val       = parameters["capacity_factor"]
AP_val       = parameters["auxiliary_power"]

# Constants
CT_el_val  = 864
CT_n_val   = 7/303.3
W_en_val   = 3 * 0.3
OF_val     = 0
DW_val     = 450

par_dict = { P_ne: P_ne_val,
             PIratio: PIratio_val,
             LT: LT_val,
             SR_m: SR_m_val,
             SR_e: SR_e_val,
             D: D_val,
             C_S: C_S_val,
             C_C: C_C_val,
             DM: DM_val,
             CP: CP_val,
             CF: CF_val,
             AP: AP_val,
             CT_el: CT_el_val,
             CT_n: CT_n_val,
             W_en: W_en_val,
             OF: OF_val,
             DW: DW_val}

# Groups of environmental categories

In [60]:
group1 = ["climate change total"]

group2 = ["carcinogenic effects", "ionising radiation", "non-carcinogenic effects", 
          "ozone layer depletion", "photochemical ozone creation", "respiratory effects, inorganics",
          "freshwater and terrestrial acidification", "freshwater ecotoxicity", 
          "freshwater eutrophication", "marine eutrophication", "terrestrial eutrophication",
          "minerals and metals", "dissipated water", "fossils", "land use"]

# Simplified equations

In [61]:
# Define symbols
alpha_1, alpha_2, beta_1, beta_2, beta_3, beta_4, beta_5, beta_6 = \
symbols('alpha_1, alpha_2, beta_1, beta_2, beta_3, beta_4, beta_5, beta_6')

## Group 1 (Climate Change)

In [62]:
eq_group1_all_thresholds = alpha_1*E_co2 + alpha_2
eq_group1_all_thresholds

E_co2*alpha_1 + alpha_2

## Group 2 

### Threshold 15/20%

In [83]:
eq_group2_15_20 = (W_d*beta_1 + beta_2)/CW_ne + W_d*beta_3 + beta_4
eq_group2_15_20

W_d*beta_3 + beta_4 + (W_d*beta_1 + beta_2)/CW_ne

## Threshold 10%

In [64]:
eq_group2_10 = (D_i*W_d*beta_1 + D_i*beta_2 + W_d*beta_3 + beta_4) / (CW_ne) + W_d*beta_5 + beta_6
eq_group2_10 

W_d*beta_5 + beta_6 + (D_i*W_d*beta_1 + D_i*beta_2 + W_d*beta_3 + beta_4)/CW_ne

## Threshold 5%

In [65]:
eq_group2_05 = (D_i*SR_p*W_d*beta_1 + D_i*SR_p*beta_2 + SR_p*beta_3 + W_d*beta_4) / (CW_ne*SR_p) + W_d*beta_5 + beta_6
eq_group2_05 

W_d*beta_5 + beta_6 + (D_i*SR_p*W_d*beta_1 + D_i*SR_p*beta_2 + SR_p*beta_3 + W_d*beta_4)/(CW_ne*SR_p)

# Compute alphas 

## All thresholds

In [66]:
# Replace parameters in equation
eq_alpha = eq.subs(par_dict)
repl = {SR_p: SR_p_val,
        D_i: D_i_val,
        CW_ne: CW_ne_val,
        W_d: W_d_val}
eq_alpha = eq_alpha.subs(repl)

alpha_dict = {}
for method in ILCD:
    is_ = dict(coeff_matrix.T[method])
    is_dict = {k.replace('.', '_'): v for k,v in is_.items()}
    is_dict['i5_1'] = 0
    is_dict['i5_2'] = 0
    if method[2] in group1:
        eq_alpha_is = eq_alpha.subs(is_dict)
        alpha1 = collect(eq_alpha_is, E_co2, evaluate=False)[E_co2]
        alpha2 = collect(eq_alpha_is, E_co2, evaluate=False)[1]
        alpha_dict[method] = {'alpha1': alpha1, 'alpha2': alpha2}
           

In [67]:
# Example of the formula
eq_alpha_is

1.0*E_co2 + 0.00376280126553773

In [68]:
alpha_df = pd.DataFrame.from_dict(alpha_dict, orient='index')
alpha_df = alpha_df.astype(float)
alpha_df_5 = alpha_df_10 = alpha_df_15 = alpha_df_20 = alpha_df
alpha_df

,,,alpha1,alpha2
ILCD 2.0 2018 midpoint no LT,climate change,climate change total,1.0,0.003763


# Compute betas

## Threshold 15-20%

In [69]:
# Threshold = 15%/20%
eq_beta = eq.subs(par_dict)
eq_beta = eq_beta.subs(E_co2, E_co2_val)
repl = {SR_p: SR_p_val, D_i: D_i_val} 
eq_beta = eq_beta.subs(repl)
        
beta_dict_15 = {}     
for method in ILCD:
    is_ = dict(coeff_matrix.T[method])
    is_dict = {k.replace('.', '_'): v for k,v in is_.items()}
    is_dict['i5_1'] = 0
    is_dict['i5_2'] = 0
    if method[2] in group2:
        eq_beta_is = eq_beta.subs(is_dict)
        eq_beta_is = ratsimp(eq_beta_is)
        temp = collect(eq_beta_is, [W_d,1/CW_ne], evaluate=False)
        beta1 = collect(temp[list(temp.keys())[1]], W_d, evaluate=False)[W_d]
        beta2 = collect(temp[list(temp.keys())[1]], W_d, evaluate=False)[1]
        beta3 = temp[W_d]
        beta4 = temp[1]
        
        beta_dict_15[method] = {'beta1': beta1, 'beta2': beta2,
                                     'beta3': beta3, 'beta4': beta4}


beta_dict_20 = beta_dict_15


In [70]:
# Show example of the formula
eq_beta_is

4.03637346737897e-13*W_d + 3.40479182493226e-8 + 1.0*(9.41820475721763e-11*W_d + 1.50348380466558e-7)/CW_ne

In [71]:
beta_df_15 = pd.DataFrame.from_dict(beta_dict_15, orient='index')
beta_df_15 = beta_df_15.astype(float)

#20% and 15% threshold have the same configuration
beta_df_20 = beta_df_15

beta_df_15

beta1  \
ILCD 2.0 2018 midpoint no LT human health      carcinogenic effects                      1.362929e-12   
                                               ionising radiation                        2.375444e-07   
                                               non-carcinogenic effects                  3.684940e-12   
                                               ozone layer depletion                     7.749298e-13   
                                               photochemical ozone creation              6.153338e-08   
                                               respiratory effects, inorganics           3.104183e-13   
                             ecosystem quality freshwater and terrestrial acidification  5.247499e-08   
                                               freshwater ecotoxicity                    4.149237e-05   
                                               freshwater eutrophication                 2.680857e-10   
                                               marine eutrophication                     1.977236e-08   
                                               terrestrial eutrophication                2.146504e-07   
                             resources         dissipated water                          1.617240e-06   
                                               fossils                                   8.367434e-05   
                                               land use                                  2.259878e-05   
                                               minerals and metals                       9.418205e-11   

                                                                                                beta2  \
ILCD 2.0 2018 midpoint no LT human health      carcinogenic effects                      9.488921e-10   
                                               ionising radiation                        1.188121e-04   
                                               non-carcinogenic effects                  2.273493e-09   
                                               ozone layer depletion                     2.936230e-10   
                                               photochemical ozone creation              2.124911e-05   
                                               respiratory effects, inorganics           3.845280e-10   
                             ecosystem quality freshwater and terrestrial acidification  2.583014e-05   
                                               freshwater ecotoxicity                    1.205226e-02   
                                               freshwater eutrophication                 2.156013e-07   
                                               marine eutrophication                     5.226224e-06   
                                               terrestrial eutrophication                5.901321e-05   
                             resources         dissipated water                          1.787674e-03   
                                               fossils                                   6.014959e-02   
                                               land use                                  3.105780e-02   
                                               minerals and metals                       1.503484e-07   

                                                                                                beta3  \
ILCD 2.0 2018 midpoint no LT human health      carcinogenic effects                      5.841125e-15   
                                               ionising radiation                        1.018047e-09   
                                               non-carcinogenic effects                  1.579260e-14   
                                               ozone layer depletion                     3.321128e-15   
                                               photochemical ozone creation              2.637145e-10   
                                               respiratory effects, inorganics           1.330364e-15   

## Threshold = 10%

In [72]:
eq_beta_10 = eq.subs(par_dict)
eq_beta_10 = eq_beta_10.subs(E_co2, E_co2_val)
repl = {SR_p: SR_p_val} 
eq_beta_10 = eq_beta_10.subs(repl)
        
beta_dict_10 = {}   
    
for method in ILCD:
    is_ = dict(coeff_matrix.T[method])
    is_dict = {k.replace('.', '_'): v for k,v in is_.items()}
    is_dict['i5_1'] = 0
    is_dict['i5_2'] = 0
    if method[2] in group2:
        eq_beta_10_is = eq_beta_10.subs(is_dict)
        eq_beta_10_is = ratsimp(eq_beta_10_is)
               
        temp   = collect(eq_beta_10_is, [W_d, 1/CW_ne,], evaluate=False)
        temp_2 = collect(temp[list(temp.keys())[1]], [D_i*W_d, D_i, W_d], evaluate=False)
        
        beta1  = temp_2[D_i*W_d]      
        beta2  = temp_2[D_i]
        beta3  = temp_2[W_d]
        beta4  = temp_2[1]
              
        beta5 = temp[W_d]
        beta6 = temp[1]
        
        beta_dict_10[method] = {'beta1': beta1, 'beta2': beta2,
                                     'beta3': beta3, 'beta4': beta4,
                                     'beta5': beta5, 'beta6': beta6}      
    

In [73]:
# Example of the formula
eq_beta_10_is

4.03637346737897e-13*W_d + 3.40479182493226e-8 + 1.0*(9.41820475721763e-10*D_i*W_d + 1.50348380466558e-6*D_i + 4.70910237860882e-11*W_d + 7.51741902332789e-8)/CW_ne

In [74]:
beta_df_10 = pd.DataFrame.from_dict(beta_dict_10, orient='index')
beta_df_10 = beta_df_10.astype(float)
beta_df_10

beta1  \
ILCD 2.0 2018 midpoint no LT human health      carcinogenic effects                      1.362929e-11   
                                               ionising radiation                        2.375444e-06   
                                               non-carcinogenic effects                  3.684940e-11   
                                               ozone layer depletion                     7.749298e-12   
                                               photochemical ozone creation              6.153338e-07   
                                               respiratory effects, inorganics           3.104183e-12   
                             ecosystem quality freshwater and terrestrial acidification  5.247499e-07   
                                               freshwater ecotoxicity                    4.149237e-04   
                                               freshwater eutrophication                 2.680857e-09   
                                               marine eutrophication                     1.977236e-07   
                                               terrestrial eutrophication                2.146504e-06   
                             resources         dissipated water                          1.617240e-05   
                                               fossils                                   8.367434e-04   
                                               land use                                  2.259878e-04   
                                               minerals and metals                       9.418205e-10   

                                                                                                beta2  \
ILCD 2.0 2018 midpoint no LT human health      carcinogenic effects                      9.488921e-09   
                                               ionising radiation                        1.188121e-03   
                                               non-carcinogenic effects                  2.273493e-08   
                                               ozone layer depletion                     2.936230e-09   
                                               photochemical ozone creation              2.124911e-04   
                                               respiratory effects, inorganics           3.845280e-09   
                             ecosystem quality freshwater and terrestrial acidification  2.583014e-04   
                                               freshwater ecotoxicity                    1.205226e-01   
                                               freshwater eutrophication                 2.156013e-06   
                                               marine eutrophication                     5.226224e-05   
                                               terrestrial eutrophication                5.901321e-04   
                             resources         dissipated water                          1.787674e-02   
                                               fossils                                   6.014959e-01   
                                               land use                                  3.105780e-01   
                                               minerals and metals                       1.503484e-06   

                                                                                                beta3  \
ILCD 2.0 2018 midpoint no LT human health      carcinogenic effects                      6.814646e-13   
                                               ionising radiation                        1.187722e-07   
                                               non-carcinogenic effects                  1.842470e-12   
                                               ozone layer depletion                     3.874649e-13   
                                               photochemical ozone creation              3.076669e-08   
                                               respiratory effects, inorganics           1.552092e-13   

## Threshold = 5%

In [75]:
eq_beta_5 = eq.subs(par_dict)
eq_beta_5 = eq_beta_5.subs(E_co2, E_co2_val)
        
beta_dict_5 = {}       
for method in ILCD:
    is_ = dict(coeff_matrix.T[method])
    is_dict = {k.replace('.', '_'): v for k,v in is_.items()}
    is_dict['i5_1'] = 0
    is_dict['i5_2'] = 0
    if method[2] in group2:
        eq_beta_5_is = eq_beta_5.subs(is_dict)
        eq_beta_5_is = ratsimp(eq_beta_5_is)
               
        temp   = collect(eq_beta_5_is, [W_d, 1/CW_ne*SR_p], evaluate=False)
        temp_2 = collect(temp[list(temp.keys())[1]], [SR_p*W_d*D_i, D_i*SR_p, SR_p, W_d], evaluate=False)
        
        beta1  = temp_2[D_i*SR_p*W_d]
        beta2  = temp_2[D_i*SR_p]
        beta3  = temp_2[W_d]
        beta4  = temp_2[SR_p]
               
        beta5 = temp[W_d]
        beta6 = temp[1]
        
        beta_dict_5[method] = {'beta1': beta1, 'beta2': beta2,
                                     'beta3': beta3, 'beta4': beta4,
                                     'beta5': beta5, 'beta6': beta6}      
    

In [76]:
# Show example of the formula
eq_beta_5_is

4.03637346737897e-13*W_d + 3.40479182493226e-8 + 1.0*(9.41820475721763e-10*D_i*SR_p*W_d + 1.50348380466558e-6*D_i*SR_p + 7.51741902332789e-8*SR_p + 4.70910237860882e-9*W_d)/(CW_ne*SR_p)

In [77]:
beta_df_5 = pd.DataFrame.from_dict(beta_dict_5, orient='index')
beta_df_5 = beta_df_5.astype(float)
beta_df_5

beta1  \
ILCD 2.0 2018 midpoint no LT human health      carcinogenic effects                      1.362929e-11   
                                               ionising radiation                        2.375444e-06   
                                               non-carcinogenic effects                  3.684940e-11   
                                               ozone layer depletion                     7.749298e-12   
                                               photochemical ozone creation              6.153338e-07   
                                               respiratory effects, inorganics           3.104183e-12   
                             ecosystem quality freshwater and terrestrial acidification  5.247499e-07   
                                               freshwater ecotoxicity                    4.149237e-04   
                                               freshwater eutrophication                 2.680857e-09   
                                               marine eutrophication                     1.977236e-07   
                                               terrestrial eutrophication                2.146504e-06   
                             resources         dissipated water                          1.617240e-05   
                                               fossils                                   8.367434e-04   
                                               land use                                  2.259878e-04   
                                               minerals and metals                       9.418205e-10   

                                                                                                beta2  \
ILCD 2.0 2018 midpoint no LT human health      carcinogenic effects                      9.488921e-09   
                                               ionising radiation                        1.188121e-03   
                                               non-carcinogenic effects                  2.273493e-08   
                                               ozone layer depletion                     2.936230e-09   
                                               photochemical ozone creation              2.124911e-04   
                                               respiratory effects, inorganics           3.845280e-09   
                             ecosystem quality freshwater and terrestrial acidification  2.583014e-04   
                                               freshwater ecotoxicity                    1.205226e-01   
                                               freshwater eutrophication                 2.156013e-06   
                                               marine eutrophication                     5.226224e-05   
                                               terrestrial eutrophication                5.901321e-04   
                             resources         dissipated water                          1.787674e-02   
                                               fossils                                   6.014959e-01   
                                               land use                                  3.105780e-01   
                                               minerals and metals                       1.503484e-06   

                                                                                                beta3  \
ILCD 2.0 2018 midpoint no LT human health      carcinogenic effects                      6.814646e-11   
                                               ionising radiation                        1.187722e-05   
                                               non-carcinogenic effects                  1.842470e-10   
                                               ozone layer depletion                     3.874649e-11   
                                               photochemical ozone creation              3.076669e-06   
                                               respiratory effects, inorganics           1.552092e-11   

# Save to excel

In [78]:
if success_rate == True:
    with pd.ExcelWriter(os.path.join(path, 'generated_files\Simplified models coefficients conventional - symbolic - thresholds.xlsx')) as writer:
        alpha_df_5.to_excel(writer, sheet_name='alpha_5%')
        alpha_df_10.to_excel(writer, sheet_name='alpha_10%')
        alpha_df_15.to_excel(writer, sheet_name='alpha_15%')
        alpha_df_20.to_excel(writer, sheet_name='alpha_20%')
        beta_df_5.to_excel(writer, sheet_name='beta_5%')
        beta_df_10.to_excel(writer, sheet_name='beta_10%')
        beta_df_15.to_excel(writer, sheet_name='beta_15%')
        beta_df_20.to_excel(writer, sheet_name='beta_20%')

## Other save

In [79]:
acronyms_dict = {
    'climate change total' : 'CC',
    'carcinogenic effects' : 'HT-c',
    'ionising radiation' : 'IR' ,
    'non-carcinogenic effects' : 'HT-nc',
    'ozone layer depletion' : 'OD',
    'photochemical ozone creation': 'POC',
    'respiratory effects, inorganics': 'RI',
    'freshwater and terrestrial acidification':'A',
    'freshwater ecotoxicity':'ET',
    'freshwater eutrophication':'Ef',
    'marine eutrophication':'Em',
    'terrestrial eutrophication':'Et',
    'dissipated water':'RUw',
    'fossils':'RUe',
    'land use':'LU',
    'minerals and metals': 'RUm'}

In [80]:
beta_dict_15_ac=dict((acronyms_dict[k[2]], v) for (k, v) in beta_dict_15.items())
beta_df_15_ac = pd.DataFrame.from_dict(beta_dict_15_ac, orient="index")
beta_df_15_ac

,beta1,beta2,beta3,beta4
HT-c,1.36292921806014e-12,9.48892081045976e-10,5.84112522025774e-15,7.54855227564241e-11
IR,2.37544384040280e-7,0.000118812088398227,1.01804736017263e-9,2.05776040402796e-5
HT-nc,3.68493979896912e-12,2.27349270518730e-9,1.57925991384391e-14,2.83441336515349e-10
OD,7.74929799532725e-13,2.93623044041408e-10,3.32112771228309e-15,7.11441973305494e-11
POC,6.15333774702975e-8,2.12491133072005e-5,2.63714474872703e-10,2.70764235388611e-6
RI,3.10418311845657e-13,3.84528047336498e-10,1.33036419362425e-15,4.51129624007364e-11
A,5.24749932896689e-8,2.58301402935724e-5,2.24892828384295e-10,4.40718712112366e-6
ET,4.14923746598420e-5,0.0120522577361756,1.77824462827894e-7,0.00132025012634752
Ef,2.68085664750245e-10,2.15601333353470e-7,1.14893856321533e-12,3.73815261835922e-8
Em,1.97723598225402e-8,5.22622355666637e-6,8.47386849537439e-11,8.32014656467570e-7


In [81]:
alpha_dict_ac=dict((acronyms_dict[k[2]], v) for (k, v) in alpha_dict.items())
alpha_df_ac = pd.DataFrame.from_dict(alpha_dict_ac, orient="index")
alpha_df_ac

,alpha1,alpha2
CC,1.00000000000000,0.00376280126553773


In [82]:
if success_rate == False:
    file_name = 'Simplified conventional coefficients acronyms - NO SR.xlsx'
elif success_rate == True:
    file_name = 'Simplified conventional coefficients acronyms.xlsx'

with pd.ExcelWriter(os.path.join(path, file_name)) as writer:
    beta_df_15_ac.to_excel(writer, sheet_name='beta_15%')
    alpha_df_ac.to_excel(writer, sheet_name='alpha')